In [ ]:
from google.colab import drive
from google.colab import files
import os

drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/VRDL/hw3')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/VRDL/hw3')
os.getcwd()

'/content/drive/MyDrive/VRDL/hw3'

In [ ]:
!pip install mmcv
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install pyyaml==5.1

In [ ]:
# https://mmdetection.readthedocs.io/en/latest/2_new_data_model.html
# make custom dataset to coco cormat
import torch
import os.path as osp
import os
import mmcv
import json
from PIL import Image
import pycocotools._mask as _mask
import pycocotools.mask as mask_util
import numpy as np
from tqdm import tqdm
import re
import cv2
from detectron2.structures import polygons_to_bitmask

class RLE():
    def encode(self, bimask):
        bimask = np.asfortranarray(bimask)  
        if len(bimask.shape) == 3:
            return _mask.encode(bimask)
        elif len(bimask.shape) == 2:
            h, w = bimask.shape
            return _mask.encode(bimask.reshape((h, w, 1), order='F'))[0]
    def decode(self, rleObjs):
        if type(rleObjs) == list:
            return _mask.decode(rleObjs)
        else:
            return _mask.decode([rleObjs])[:,:,0]

    def area(self, rleObjs):
        if type(rleObjs) == list:
            return _mask.area(rleObjs)
        else:
            return _mask.area([rleObjs])[0]

    def toBbox(self, rleObjs):
        if type(rleObjs) == list:
            return _mask.toBbox(rleObjs)
        else:
            return _mask.toBbox([rleObjs])[0]
    def polygonFromMask(self, maskedArr): # https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py

        contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        segmentation = []
        for contour in contours:
            # Valid polygons have >= 6 coordinates (3 points)
            if contour.size >= 6:
                segmentation.append(contour.flatten().tolist())
        RLEs = mask_util.frPyObjects(segmentation, maskedArr.shape[0], maskedArr.shape[1])
        RLE = mask_util.merge(RLEs)
        # RLE = mask.encode(np.asfortranarray(maskedArr))
        area = mask_util.area(RLE)
        [x, y, w, h] = cv2.boundingRect(maskedArr)

        return segmentation[0] #, [x, y, w, h], area

    def polygon_to_rle(self, polygon: list, shape=(1000, 1000)):
        '''
        polygon: a list of [x1, y1, x2, y2,....]
        shape: shape of bitmask
        Return: RLE type of mask
        '''
        mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1]) # add 0.25 can keep the pixels before and after the conversion unchanged
        rle = mask_util.encode(np.asfortranarray(mask))
        return rle

rle = RLE()

def convert_nuclei_to_coco(is_train=True):
    if is_train:
        mode = 'train'
    else:
        mode = 'test'

    images = []
    annotations = []
    anno_id = 0
    image_filename_list = os.listdir(osp.join('dataset', mode))
    image_filename_list = list(filter(lambda x: re.match('TCGA', x) != None, image_filename_list))
     
    for idx, filename in enumerate(image_filename_list):
        mask_filename_list = os.listdir(osp.join('dataset', mode, filename, 'masks'))
        mask_filename_list = list(filter(lambda x: re.match('.+\.png', x) != None, mask_filename_list))

        img_path = osp.join('dataset', mode, filename, 'images', filename + '.png')
        height, width = mmcv.imread(img_path).shape[:2]
        # For image
        images.append(dict(
            id = idx,
            file_name = filename+'.png',
            height = height,
            width = width))
        
        # For annotations
        for mask_name in tqdm(mask_filename_list):

            mask_path = osp.join('dataset', mode, filename, 'masks', mask_name)
            mask_img = mmcv.imread(mask_path, 0)
            poly = rle.polygonFromMask(mask_img)
            rle_code = rle.encode(mask_img)
            rle_code['counts'] = rle_code['counts'].decode('ascii')
            # just use first channel to encode
            # because the matrix are equal for each channels with each other
            rle_box = list(rle.toBbox(rle_code).tolist())# [x, y, width, height]
            rle_area = int(rle.area(rle_code))

            annotations.append(dict(
                    id = anno_id,
                    image_id = idx,
                    category_id = 1,
                    iscrowd = 0,
                    segmentation = [poly],
                    bbox = rle_box,
                    area = rle_area
                )
            )
            anno_id += 1

    categories=[{'id':1, 'name': 'cell', 'supercategory':'cell'}]
    coco_format_json = dict(
        images = images,
        annotations = annotations,
        categories=categories)
    return coco_format_json

coco_format_json = convert_nuclei_to_coco()

#save to json
with open('train_info.json', 'w') as f:
    json.dump(coco_format_json, f, indent=4)


100%|██████████| 328/328 [00:03<00:00, 109.04it/s]


In [ ]:
coco_format_json

{'annotations': [{'area': 147,
   'bbox': [455.0, 208.0, 11.0, 17.0],
   'category_id': 1,
   'id': 0,
   'image_id': 0,
   'iscrowd': 0,
   'segmentation': [[459,
     208,
     458,
     209,
     457,
     209,
     456,
     210,
     456,
     212,
     455,
     213,
     455,
     222,
     457,
     224,
     459,
     224,
     460,
     223,
     461,
     223,
     463,
     221,
     463,
     219,
     465,
     217,
     465,
     212,
     464,
     211,
     464,
     210,
     463,
     209,
     462,
     209,
     461,
     208]]},
  {'area': 51,
   'bbox': [374.0, 83.0, 9.0, 10.0],
   'category_id': 1,
   'id': 1,
   'image_id': 0,
   'iscrowd': 0,
   'segmentation': [[378,
     83,
     376,
     85,
     376,
     86,
     375,
     87,
     375,
     88,
     374,
     89,
     374,
     92,
     375,
     92,
     379,
     88,
     380,
     88,
     382,
     86,
     382,
     84,
     381,
     83]]},
  {'area': 116,
   'bbox': [430.0, 787.0, 11.0, 13.0],
  